# import and read

In [137]:
import pandas as pd
import os
import sqlalchemy as sql
import numpy as np
import sqlite3

In [138]:
# Pfad zur Excel-Datei
file_path = 'C:\\Users\\hp\\OneDrive\\Desktop\\DBU\\SP3-1.Dateninfrastrukturen\\Pruefung\\Zensus\\input\\Zensus.xlsx'
# Lese die Excel-Datei ein und speichere die Blätter als einzelne DataFrames
sheets = pd.read_excel(file_path, sheet_name=None)
data_frames = {sheet_name: data for sheet_name, data in sheets.items() if sheet_name.startswith('CSV')}

# check
print("Dateninfrastruktur aufgesetzt und Excel-Daten geladen.")

Dateninfrastruktur aufgesetzt und Excel-Daten geladen.


In [139]:
# Speichern der DataFrames in separaten Variablen mit angepassten Namen
created_dfs = {}
for sheet_name, data in data_frames.items():
    # Ersetzen von '-' und '/' durch '_' in den Tabellenblätternamen
    new_sheet_name = sheet_name.replace('-', '_').replace('/', '_')
    
    # Ersetzen von '.', '/', '-' und '–' durch 0 in den Daten
    df = data.apply(lambda x: x.map(lambda y: 0 if y in ['.', '/', '-', '–'] else y))
    
    # Bereinigung der 'Regionalebene'-Spalte (Kreise)
    if 'Regionalebene' in df.columns:
        df['Regionalebene'] = df['Regionalebene'].str.replace('/', '')
        df['Regionalebene'] = df['Regionalebene'].str.replace(' ', '')
    # Umwandlung in numerische Typen für Spalten ab der 5.
    for col in df.columns[4:]:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
    
    globals()[new_sheet_name] = df
    created_dfs[new_sheet_name] = df

# Anzeigen der neuen Namen der erstellten DataFrames
print("Erstellte DataFrames:")
for df_name in created_dfs.keys():
    print(df_name)

Erstellte DataFrames:
CSV_Klassenstufe
CSV_Schulform
CSV_Hoechster_Schulabschluss
CSV_Hoechster_berufl_Abschluss
CSV_Erwerbsstatus
CSV_ET_Alter
CSV_ET_Hoechst_berufl_Abschl
CSV_ET_Stellung_im_Beruf
CSV_ET_Beruf_Hauptgr_ISCO08
CSV_ET_Wirtschaftszweig


## EDA

In [140]:
CSV_Klassenstufe.groupby('Regionalebene')['SCHUELER_KLSS_STP'].sum()

Regionalebene
Bund                                  8903780
Gemeinde                              6631640
Gemeindeverband                       1310750
Land                                  8903780
Regierungsbezirk                      7013880
StadtkreiskreisfreieStadtLandkreis    8903770
Name: SCHUELER_KLSS_STP, dtype: int64

In [141]:
Berichtszeitpunkt = CSV_ET_Alter['Berichtszeitpunkt'].unique()[0]
Berichtszeitpunkt

np.int64(20220515)

In [142]:
# Angenommen, df ist Ihr DataFrame mit der Spalte 'Name'
df = created_dfs['CSV_ET_Alter']  # Passen Sie dies an, um den entsprechenden DataFrame zu verwenden

# Extrahieren der Werte nach dem Komma in der Spalte 'Name'
df['Name_suffix'] = df['Name'].str.split(',').str[1].str.strip()

# Einzigartige Werte in der neuen Spalte 'Name_suffix' anzeigen
unique_suffixes = df['Name_suffix'].unique()

print("Einzigartige Werte nach dem Komma in der Spalte 'Name':")
print(unique_suffixes)

Einzigartige Werte nach dem Komma in der Spalte 'Name':
[nan 'Stadt' 'Landeshauptstadt' 'Hansestadt' 'Kirchspiel'
 'Freie und Hansestadt' 'Flecken' 'Berg- und Universitätsstadt'
 'Münchhausenstadt' 'Klostergemeinde' 'gemfr. Bezirk' 'Klosterflecken'
 'Inselgemeinde' 'Nordseebad' 'Klingenstadt' 'Fundort des Neanderthalers'
 'Burggemeinde' 'Sport- und Freizeitgemeinde' 'Tor zur Eifel'
 'Kupferstadt' 'Kolpingstadt' 'Schloss-Stadt' 'Stadt auf der Höhe'
 'Blütenstadt' 'Glockenstadt' 'Stadt der Burgmannshöfe' 'NRW-Klimakommune'
 'Die Mähdrescherstadt' 'Widukindstadt' 'Orgelstadt'
 'Stadt der Osterräder' 'Sennegemeinde' 'Stadt der FernUniversität'
 'Stadt der Kluterthöhle' 'Kreis- und  Hochschulstadt' 'Universitätsstadt'
 'Hansestadt an der Ruhr' 'Wissenschaftsstadt' 'Kreisstadt'
 'Karolingerstadt' 'Schöfferstadt' 'Büchnerstadt' 'Barbarossastadt'
 'Brüder-Grimm-Stadt' 'Einhardstadt' 'Hochschulstadt'
 'Friedrich-Ludwig-Weidig-Stadt' 'Oranienstadt' 'Marktflecken'
 'documenta-Stadt' 'Marktgemeind

## Transform

In [143]:
import pandas as pd

# Neue Tabelle für Stadt-Typen erstellen
stadt_typen = pd.DataFrame(columns=['Schluessel', 'Typ'])

# Bereinigung der DataFrames und Auslagern der Stadt-Typen
for name, df in created_dfs.items():
    # Extrahieren des Stadtnamens und Typs
    df['Stadt'] = df['Name'].str.split(',').str[0]
    df['Typ'] = df['Name'].str.split(',').str[1].str.strip()
    
    # Kombinieren in die neue Tabelle
    temp_df = df[['_RS', 'Typ']].drop_duplicates().rename(columns={'_RS': 'Schluessel'})
    stadt_typen = pd.concat([stadt_typen, temp_df])
    
    # Bereinigen der 'Name'-Spalte
    df['Name'] = df['Stadt']
    
    # Entfernen der temporären Spalten
    df = df.drop(columns=['Stadt', 'Typ'])
    
    # Aktualisieren des DataFrames im Dictionary
    created_dfs[name] = df

# Entfernen von Duplikaten und NaN-Werten in der neuen Tabelle
stadt_typen = stadt_typen.drop_duplicates().dropna().reset_index(drop=True)

# Neue Tabelle und bereinigte DataFrames anzeigen
print("Stadt-Typen Tabelle:")
print(stadt_typen.head())

print("Bereinigte DataFrames:")
for name, df in created_dfs.items():
    print(f"{name}:")
    print(df.head())

Stadt-Typen Tabelle:
    Schluessel               Typ
0         1001             Stadt
1  10010000000             Stadt
2         1002  Landeshauptstadt
3  10020000000  Landeshauptstadt
4         1003        Hansestadt
Bereinigte DataFrames:
CSV_Klassenstufe:
   Berichtszeitpunkt          _RS                Name  \
0           20220515            0         Deutschland   
1           20220515            1  Schleswig-Holstein   
2           20220515         1001           Flensburg   
3           20220515  10010000000           Flensburg   
4           20220515         1002                Kiel   

                        Regionalebene  SCHUELER_KLSS_STP  \
0                                Bund            8903780   
1                                Land             320060   
2  StadtkreiskreisfreieStadtLandkreis              10380   
3                            Gemeinde              10380   
4  StadtkreiskreisfreieStadtLandkreis              24340   

   SCHUELER_KLSS_STP__M  SCHUELER_KL

In [144]:
stadt_typen.Schluessel.value_counts()

Schluessel
160775052043    1
1001            1
10010000000     1
1002            1
160745014041    1
               ..
10040000000     1
10510011011     1
10510044044     1
10515166072     1
10515175074     1
Name: count, Length: 2628, dtype: int64

## regionalebenen

In [145]:
# Definieren der Regionalebenen
liste = [
    'Bund',
    'Gemeinde',
    'Gemeindeverband',
    'Land',
    'Regierungsbezirk',
    'StadtkreiskreisfreieStadtLandkreis'
]

In [146]:

# Erstellen der Tabellen für alle Regionalebenen
regionalebenen = {}
for ebene in liste:
    regionalebenen[ebene] = pd.DataFrame()
    for df in created_dfs.values():
        if 'Regionalebene' in df.columns:
            filtered_df = df[df['Regionalebene'] == ebene][['_RS', 'Name']]
            regionalebenen[ebene] = pd.concat([regionalebenen[ebene], filtered_df], ignore_index=True)
    regionalebenen[ebene] = regionalebenen[ebene].drop_duplicates().reset_index(drop=True)

    # check
    print(f"Tabelle für {ebene} wurde erfolgreich erstellt.")
    print(regionalebenen[ebene].head())

Tabelle für Bund wurde erfolgreich erstellt.
   _RS         Name
0    0  Deutschland
Tabelle für Gemeinde wurde erfolgreich erstellt.
           _RS         Name
0  10010000000    Flensburg
1  10020000000         Kiel
2  10030000000       Lübeck
3  10040000000   Neumünster
4  10510011011  Brunsbüttel
Tabelle für Gemeindeverband wurde erfolgreich erstellt.
        _RS                    Name
0  10515163  Burg-St. Michaelisdonn
1  10515166           Marne-Nordsee
2  10515169                   Eider
3  10515172           Heider Umland
4  10515175      Mitteldithmarschen
Tabelle für Land wurde erfolgreich erstellt.
   _RS                 Name
0    1   Schleswig-Holstein
1    2              Hamburg
2    3        Niedersachsen
3    4               Bremen
4    5  Nordrhein-Westfalen
Tabelle für Regierungsbezirk wurde erfolgreich erstellt.
   _RS                              Name
0   31  Statistische Region Braunschweig
1   32      Statistische Region Hannover
2   33      Statistische Region L

In [147]:
# Erstellen der Spalten für Gemeindeverbandsschlüssel (erste 8 Stellen)
for ebene in ['Gemeinde', 'Gemeindeverband']:
    regionalebenen[ebene]['Gemeindeverbandsschluessel'] = regionalebenen[ebene]['_RS'].astype(str).str[:8].astype(int)

# Erstellen der Spalten für Kreisschlüssel (erste 4 Stellen)
for ebene in ['Gemeindeverband', 'StadtkreiskreisfreieStadtLandkreis']:
    regionalebenen[ebene]['Kreisschluessel'] = regionalebenen[ebene]['_RS'].astype(str).str[:5].astype(int)

# Erstellen der Spalten für Landesschlüssel (erste 2 Stellen)
for ebene in ['StadtkreiskreisfreieStadtLandkreis', 'Regierungsbezirk']:
    regionalebenen[ebene]['RegBezirkschluessel'] = regionalebenen[ebene]['_RS'].astype(str).str[:3].astype(int)

# Erstellen der Spalten für Landesschlüssel (erste 2 Stellen)
for ebene in ['Regierungsbezirk', 'Land']:
    regionalebenen[ebene]['Landesschluessel'] = regionalebenen[ebene]['_RS'].astype(str).str[:2].astype(int)

# Erstellen der Spalten für Landesschlüssel (erste 2 Stellen)
for ebene in ['Land', 'Bund']:
    regionalebenen[ebene]['Bundschluessel'] = regionalebenen[ebene]['_RS'].astype(str).str[:1].astype(int)

# Alle Tabellen ohne die Spalte '_RS' speichern (ausser Gemeinde)
for ebene, df in regionalebenen.items():
    if ebene != 'Gemeinde':
        regionalebenen[ebene] = df.drop('_RS', axis=1)

# _RS in Gemeindeschlüssel umbenennen
regionalebenen['Gemeinde'] = regionalebenen['Gemeinde'].rename(columns={'_RS': 'Gemeindeschluessel'})


In [148]:
regionalebenen['StadtkreiskreisfreieStadtLandkreis']

,Name,Kreisschluessel,RegBezirkschluessel
0,Flensburg,1001,100
1,Kiel,1002,100
2,Lübeck,1003,100
3,Neumünster,1004,100
4,Dithmarschen,1051,105
...,...,...,...
395,Saalfeld-Rudolstadt,16073,160
396,Saale-Holzland-Kreis,16074,160
397,Saale-Orla-Kreis,16075,160
398,Greiz,16076,160


In [149]:
# Überprüfen der ersten Zeilen der Tabellen

print("Gemeindetabelle:")
print(regionalebenen['Gemeinde'].head())

print("\nGemeindeverbandstabelle:")
print(regionalebenen['Gemeindeverband'].head())

print("\nKreis:")
print(regionalebenen['StadtkreiskreisfreieStadtLandkreis'].head())

print("\nRegBezirk:")
print(regionalebenen['Regierungsbezirk'].head())

print("\nLändertabelle:")
print(regionalebenen['Land'].head())

Gemeindetabelle:
   Gemeindeschluessel         Name  Gemeindeverbandsschluessel
0         10010000000    Flensburg                    10010000
1         10020000000         Kiel                    10020000
2         10030000000       Lübeck                    10030000
3         10040000000   Neumünster                    10040000
4         10510011011  Brunsbüttel                    10510011

Gemeindeverbandstabelle:
                     Name  Gemeindeverbandsschluessel  Kreisschluessel
0  Burg-St. Michaelisdonn                    10515163            10515
1           Marne-Nordsee                    10515166            10515
2                   Eider                    10515169            10515
3           Heider Umland                    10515172            10515
4      Mitteldithmarschen                    10515175            10515

Kreis:
           Name  Kreisschluessel  RegBezirkschluessel
0     Flensburg             1001                  100
1          Kiel             1002     

## restliche tabellen 

In [150]:
# Liste erstellen fuer die uebrigen dfs
tabellen_liste = [
    'CSV_Klassenstufe',
    'CSV_Schulform',
    'CSV_Hoechster_Schulabschluss',
    'CSV_Hoechster_berufl_Abschluss',
    'CSV_Erwerbsstatus',
    'CSV_ET_Alter',
    'CSV_ET_Hoechst_berufl_Abschl',
    'CSV_ET_Stellung_im_Beruf',
    'CSV_ET_Beruf_Hauptgr_ISCO08',
    'CSV_ET_Wirtschaftszweig'
]

In [151]:
# Erstellen der Tabellen für alle definierten Tabellen und Konvertieren der Spalten

# Liste für die DataFrames
andere_tabellen = {}
# Iteration über die Liste der Tabellen
for tabelle in tabellen_liste:
    tabelle_name = tabelle.replace('CSV_', '').lower()# Ersetzen von 'CSV_' durch '' und Umwandlung in Kleinbuchstaben
    df = created_dfs[tabelle].drop(columns=['Berichtszeitpunkt', 'Name', 'Regionalebene']).drop_duplicates().reset_index(drop=True)# Entfernen der Spalten
    df['Schluessel'] = df['_RS'].astype(int)# Erstellen einer Schluessel-Spalte aus der '_RS'-Spalte zu Integer
    df = df.drop(columns=['_RS'])
    df = df.fillna(0)  # NA-Werte durch 0 ersetzen
    # Konvertieren der Spalten in numerische Typen
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].apply(lambda x: int(x) if str(x).isdigit() else 0)
        else:
            df[col] = df[col].astype(int)
    # Verschieben der Schluessel-Spalte an die erste Position
    cols = ['Schluessel'] + [col for col in df if col != 'Schluessel']
    df = df[cols]
    andere_tabellen[tabelle] = df

   
    # Anzeigen aller Tabellennamen in andere_tabellen
print("Alle Tabellennamen in andere_tabellen:")
for table_name in andere_tabellen.keys():
    print(table_name)

Alle Tabellennamen in andere_tabellen:
CSV_Klassenstufe
CSV_Schulform
CSV_Hoechster_Schulabschluss
CSV_Hoechster_berufl_Abschluss
CSV_Erwerbsstatus
CSV_ET_Alter
CSV_ET_Hoechst_berufl_Abschl
CSV_ET_Stellung_im_Beruf
CSV_ET_Beruf_Hauptgr_ISCO08
CSV_ET_Wirtschaftszweig


# export

In [152]:
# Verbindung zur SQLite-Datenbank herstellen
db_path = r'C:\Users\hp\OneDrive\Desktop\DBU\SP3-1.Dateninfrastrukturen\Pruefung\Zensus\temp\zensusdb'
if not os.path.exists(os.path.dirname(db_path)):
    raise FileNotFoundError(f"Der Pfad {os.path.dirname(db_path)} existiert nicht.")
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


In [153]:
# Alle vorhandenen Tabellen droppen
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
for table in tables:
    cursor.execute(f'DROP TABLE IF EXISTS "{table[0]}";')

In [154]:
# Regionalebenen-Tabellen in die SQLite-Datenbank übertragen
for ebene, df in regionalebenen.items():
    df.to_sql(f'dim_{ebene.lower()}', conn, if_exists='replace', index=False)

In [155]:
# Andere Tabellen in die SQLite-Datenbank übertragen
for tabelle, df in andere_tabellen.items():
    df.to_sql(f'dim_{tabelle.lower()}', conn, if_exists='replace', index=False)

In [156]:
# Stadt-Typen-Tabelle in die SQLite-Datenbank übertragen
stadt_typen.to_sql('dim_stadt_typen', conn, if_exists='replace', index=False)

2628

In [157]:
conn.close()
print("DataFrames erfolgreich in die SQLite-Datenbank übertragen.")

DataFrames erfolgreich in die SQLite-Datenbank übertragen.


# loeschen

In [158]:
# db_path = r'C:\Users\hp\OneDrive\Desktop\DBU\SP3-1.Dateninfrastrukturen\Pruefung\Zensus\temp\zensusdb'

# conn = sqlite3.connect(db_path)
# cursor = conn.cursor()

# # Deaktivieren von Foreign Key Constraints
# cursor.execute("PRAGMA foreign_keys = OFF;")

# # Abfrage aller Tabellen
# cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# tables = cursor.fetchall()

# # Generieren und Ausführen der DROP TABLE-Befehle für jede Tabelle
# for table in tables:
#     cursor.execute(f'DROP TABLE IF EXISTS "{table[0]}";')

# # Reaktivieren von Foreign Key Constraints
# cursor.execute("PRAGMA foreign_keys = ON;")

# # Änderungen speichern und Verbindung schließen
# conn.commit()
# conn.close()

# print("Alle Tabellen wurden erfolgreich gelöscht.")